<a href="https://colab.research.google.com/github/tazkiah07/web_/blob/main/orienteed%20problame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install docplex
!pip install cplex
!pip install qpsolvers

In [6]:
import numpy as np
from qpsolvers import solve_qp
import cplex
import docplex
from docplex.mp.model import Model

In [11]:
num_points = 10
depot = 1
cost_limit = 300000

# Jarak atau biaya perjalanan dari titik i ke titik j (dalam contoh ini diisi dengan angka random)
cost_matrix = [
    [0,70000,70000,65000,65000,75000,230000,65000,75000,85000],
    [70000,0,22500,12000,4250,17500,83000,6500,17000,21000],
    [70000,20500,0,14000,18000,3000,113000,15000,60000,17500],
    [65000,12000,14000,0,8000,16000,94000,4500,16500,25500],
    [65000,4250,18000,8000,0,18500,91500,3750,18500,25500],
    [75000,17500,3000,16000,18500,0,112000,15000,4000,16500],
    [23000,83000,113000,94000,91500,112000,0,90000,105000,105000],
    [65000,6500,15000,4500,3750,15000,90000,0,13500,21000],
    [75000,17000,6000,16500,18500,4000,105000,13500,0,13500],
    [85000,21000,17500,25500,25500,16500,105000,21000,13500,0]
]

# Poin minat wisata untuk setiap titik
poi_values = [0, 4.5, 4.0, 4.5, 4.6, 4.6, 4.5, 4.7, 4.8, 4.7]

# Inisialisasi model
model = Model(name='Orienteering')

# Variabel keputusan
x = {(i, j): model.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(num_points) for j in range(num_points)}
y = {i: model.binary_var(name='y_{0}'.format(i)) for i in range(num_points)}

# Fungsi tujuan: maksimalkan poin minat wisata
model.maximize(model.sum(poi_values[i] * y[i] for i in range(num_points)))

# Kendala: setiap titik harus dikunjungi tepat satu kali
for i in range(num_points):
    model.add_constraint(model.sum(x[i, j] for j in range(num_points) if j != i)== 1, ctname='visit_once_outgoing_{0}'.format(i))
    model.add_constraint(model.sum(x[j, i] for j in range(num_points) if j != i)== 1, ctname='visit_once_incoming_{0}'.format(i))

# Kendala: total biaya perjalanan tidak boleh melebihi cost_limit
model.add_constraint(model.sum(cost_matrix[i][j] * x[i, j] for i in range(num_points) for j in range(num_points)) <= cost_limit, ctname='cost_limit')

# Kendala: hubungan antara variabel x_{ij} dan y_i
for i in range(num_points):
    for j in range(1, num_points):
        model.add_constraint(y[j] >= y[i] + x[i, j] - 1, ctname='visit_order_{0}_{1}'.format(i, j))

# Solve
solution=model.solve()

# Output solusi
print(solution)
print("Objective Value:", model.objective_value)
print("\nVisited Points:")
for i in range(num_points):
    if model.solution.get_value(y[i]) > 0.5:
        print("Point", i, "- Value:",poi_values[i])

solution for: Orienteering
objective: 40.9
status: OPTIMAL_SOLUTION(2)
x_0_2=1
x_1_3=1
x_2_7=1
x_3_5=1
x_4_1=1
x_5_6=1
x_6_0=1
x_7_4=1
x_8_9=1
x_9_8=1
y_1=1
y_2=1
y_3=1
y_4=1
y_5=1
y_6=1
y_7=1
y_8=1
y_9=1

Objective Value: 40.900000000000006

Visited Points:
Point 1 - Value: 4.5
Point 2 - Value: 4.0
Point 3 - Value: 4.5
Point 4 - Value: 4.6
Point 5 - Value: 4.6
Point 6 - Value: 4.5
Point 7 - Value: 4.7
Point 8 - Value: 4.8
Point 9 - Value: 4.7
